# Introduction to SQL for Excel Users – Part 15: Basic LEFT JOINs

[Original post](https://www.daveondata.com/blog/introduction-to-sql-for-excel-users-part-15-basic-left-joins/)

## Multiple Tables in Excel – Part 1

It is a common experience in Excel to combine data from two tables.

Yes, I’m talking about the venerable VLOOKUP function.

VLOOKUP is the go-to solution when you have data in a “lookup table” in your workbook.

Take the following contrived example:

![excel lookup table](15\excelleftjoin1.png)

The situation ☝ is quite common in Excel.

Left Table approximates a table of orders and Right Table approximates a lookup table of product names.

BTW – The reasoning for the odd table names will become clear shortly! 😉

Combining the data using VLOOKUP is not so difficult:

![excel vlookup function](15\excelleftjin2.png)

If you’re not familiar, the above Excel magic in cell D4 can be interpreted as:

1. Take the value in cell B4…
1. Go over to to the virtual table of data defined by $G$4:$H$6…
1. Find the matching value…
1. Return whatever value is in column 2…
1. Oh, and perform an exact match of B4, please!

Excel happily complies with the request. When I copy the formula down, I get the following:

![combing table data with the excel vlookup function](15\excelleftjoin3.png)

Voila!

Using VLOOKUP, I have combined (what in SQL we call JOINed) data from Right Table to Left Table.

Before I can get to the sweet SQL goodness, I need to cover a couple of other ideas.

## Multiple Tables in Excel – Part 2

Take the enhanced contrived example:

![excel vlookup function with duplicate and missing values](15\excelleftjoin4.png)

This example has some “improvements”:

1. There is a ProductID of 4 in Left Table that doesn’t exist in Right Table
1. The ProductID of 1 is duplicated in Right Table

We’ll see why these are improvement in a sec.

When I copy the VLOOKUP function call down the length of Left Table:

![excel vlookup](15\excelleftjoin5.png)

The first thing worth noticing in the ☝ is the #N/A corresponding to ProductID = 4 in Left Table, specifically:

1. The data in Left Table remains in place, even though the VLOOKUP didn’t find a match
1. Since no match was found, Excel indicates an absence of data via #N/A (in SQL we call absence of data NULL)

The second thing worth noticing in the ☝ is that only the first ProductID = 1 that VLOOKUP finds in Right Table is used.

In other words, the ProductName of SM Widget never gets used. 😕

All of the ideas covered so far are important for what comes next.

Time to SQL!

# SQL Multiple Tables – Part 1

Not surprisingly, the concepts of Left Table and Right Table are critical for working with multiple tables in SQL.

It’s so important, SQL specifically uses the keywords LEFT and RIGHT!

For this post I will be using DimEmployee as the Left Table.  

![employee table](15\dimemployee.png)

As usual, I will ignore most of the ☝.

For the purposes of this post, I will be using only the CustomerKey, FirstName, and LastName columns.

Since there are more rows in DimEmployee than I need for the purposes of this blog post, I’m going to filter down to just six:

In [ ]:
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
FROM DimEmployee E 
WHERE E.EmployeeKey = 271 OR
      E.EmployeeKey = 274 OR
      E.EmployeeKey = 275 OR
      E.EmployeeKey = 277 OR
      E.EmployeeKey = 282 OR
      E.EmployeeKey = 283

The SQL ☝ can be written a little more succinctly using the IN keyword:

In [ ]:
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
FROM DimEmployee E 
WHERE E.EmployeeKey IN (271, 274, 275, 277, 282, 283)

Using IN is the equivalent of using a bunch of ORs in your SQL code.

Basically, SQL Server just converts the INs to a bunch of ORs for you when you execute the query.

Now, for the Right Table. I will be using FactSalesQuota.  

![fact sales quota table](15\factsalesquota.png)

For this post I will only be using the EmployeeKey and SalesAmountQuota columns.

In [ ]:
SELECT SQ.EmployeeKey
      ,SQ.SalesAmountQuota
FROM FactSalesQuota SQ
ORDER BY SQ.EmployeeKey, SQ.SalesQuotaKey

Notice that it is possible for a single EmployeeKey to have more than 1 SalesAmountQuota.

Aight, now we’re ready to JOIN!

## SQL LEFT JOIN

In the Excel example ☝, I demonstrated how to combine data.

Specifically, how to keep all the Left Table data and combine it, where possible, with data from the Right Table.

This is what is known in SQL as a LEFT JOIN or, as I prefer, a LEFT OUTER JOIN.

SQL supports a number of JOINs and the LEFT OUTER JOIN is one that you will use most frequently.

When working with JOINs, it is helpful to think of a progression of virtual tables from the left-most virtual table to the right most virtual table.

This is idea is easier to grok when you see it built-up with code.

Take the following SQL:

In [ ]:
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
FROM DimEmployee E 
WHERE E.EmployeeKey IN (271, 274, 275, 277, 282, 283)

When working with JOINs, I can think of the code ☝ as establishing my left-most virtual table based on DimEmployee.

Now I tell the DB I would like to perform a LEFT OUTER JOIN with FactSalesQuota:

```
-- This code will fail - no matching logic for join
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
      ,SQ.SalesAmountQuota
FROM DimEmployee E 
    LEFT OUTER JOIN FactSalesQuota SQ 
WHERE E.EmployeeKey IN (271, 274, 275, 277, 282, 283)
```

The code ☝ formally establishes DimEmployee as the Left Table and FactSalesQuota as the Right Table.

Please note that the code above is not legit. If you try and execute the code, it will fail.

The reason for this is simple – I havent defined the matching logic for the JOIN.

Remember how I had to have VLOOKUP match on ProductIDs?

You need to do the same in SQL using the ON clause:

In [ ]:
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
      ,SQ.SalesAmountQuota
FROM DimEmployee E 
    LEFT OUTER JOIN FactSalesQuota SQ ON (E.EmployeeKey = SQ.EmployeeKey)
WHERE E.EmployeeKey IN (271, 274, 275, 277, 282, 283)

The query ☝ can be thought of as logically excuting as follows:

1. Create a virtual table of all rows of DimEmployee…
1. LEFT OUTER JOIN FactSalesQuota by matching EmployeeKey values…
1. Create a new, combined virtual table from 1 & 2…
1. Keep only the rows WHERE EmployeeKey is IN the defined list…
1. SELECT the EmployeeKey, FirstName, LastName, and SalesAmountQuota columns

As with Excel’s VLOOKUP, you are not assured a match with LEFT OUTER JOIN.

In the output ☝, we see employees that do not have sales quotas have NULLs in the SalesAmountQuota column.

Again, in SQL the NULL denotes an absence of data.

Also, note that unlike VLOOKUP, SQL LEFT OUTER JOINs match every duplicate value in the Right Table and return multiple rows in the final virtual table.

Sweet!

## Beware the NULLs of SQL

When working with SQL, NULLs add an additional level of complexity that you must deal with.

The good news is that I will be gradually introducing NULL nuances throughout the series.

For this post I want to start simple – filtering based on NULL.

As I described previously, SQL processes WHERE after FROM and all JOINs.

This means we can filter on NULL values if we would like.

Let’s say I’m intersted in only the subset of employees that do not have a sales quota.

That is, where the SalesAmountQuota IS NULL.

In [ ]:
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
      ,SQ.SalesAmountQuota
FROM DimEmployee E 
    LEFT OUTER JOIN FactSalesQuota SQ ON (E.EmployeeKey = SQ.EmployeeKey)
WHERE E.EmployeeKey IN (271, 274, 275, 277, 282, 283) AND
      SQ.SalesAmountQuota IS NULL


I can’t stress this enough.

When you want to filter when values are NULL, you have to use IS NULL!

You’ve been warned. 😁

Not surprisingly, if you want to filter in the opposite you add NOT:

In [ ]:
SELECT E.EmployeeKey
      ,E.FirstName
      ,E.LastName
      ,SQ.SalesAmountQuota
FROM DimEmployee E 
    LEFT OUTER JOIN FactSalesQuota SQ ON (E.EmployeeKey = SQ.EmployeeKey)
WHERE E.EmployeeKey IN (271, 274, 275, 277, 282, 283) AND
      SQ.SalesAmountQuota IS NOT NULL

There you have it.

The basics of the mighty LEFT OUTER JOIN, including filtering on NULLs.

Now, go forth and craft SQL goodnes!

## The Learning Arc

Given the central nature in SQL, posts about JOINs will cotinue for a bit.

Next up is coverage of INNER JOINs.

Stay healthy and happy data sleuthing!